In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Importing Twitter Sentimment Dataset

In [4]:
# API
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

100%|██████████| 80.9M/80.9M [00:00<00:00, 102MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kazanova/sentiment140/versions/2


In [5]:
import os
print(os.listdir("/root/.cache/kagglehub/datasets/kazanova/sentiment140/versions/2"))


['training.1600000.processed.noemoticon.csv']


In [6]:
column=["target","ids","date","flag","user","text"]
df = pd.read_csv("/root/.cache/kagglehub/datasets/kazanova/sentiment140/versions/2/training.1600000.processed.noemoticon.csv",names=column, encoding='latin-1')

In [7]:
# df=df.set_axis(,axis=1)

In [8]:
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
df.target.value_counts()

,count
target,
0,800000
4,800000


In [10]:
df.shape


(1600000, 6)

In [11]:
df.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [12]:
# converting target4 to one to show positive
df.target=df.target.replace(4,1)

In [13]:
df.target.value_counts()

,count
target,
0,800000
1,800000


1---> Positive

0---> Negative

Stemming

In [14]:
port_stem= PorterStemmer()

In [15]:
def stemming(content):
  stemmed_content=re.sub("[^a-zA-Z]"," ",content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
  stemmed_content=" ".join(stemmed_content)
  return stemmed_content

In [16]:
df["stemmed_content"]=df["text"].apply(stemming)

In [17]:
df.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [18]:
print(df["stemmed_content"][:5])

0    switchfoot http twitpic com zl awww bummer sho...
1    upset updat facebook text might cri result sch...
2    kenichan dive mani time ball manag save rest g...
3                      whole bodi feel itchi like fire
4                        nationwideclass behav mad see
Name: stemmed_content, dtype: object


In [19]:
print(df["target"][:5])

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64


In [20]:
# separating data and label
X=df["stemmed_content"].values
Y=df["target"].values

In [21]:
print(X[0])

switchfoot http twitpic com zl awww bummer shoulda got david carr third day


In [22]:
print(Y)

[0 0 0 ... 1 1 1]


#### Splitting data to training and test data

In [23]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=0.2,stratify=Y,random_state=2)

In [24]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [26]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [27]:
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [28]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 461488)>
  Coords	Values
  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (12

In [29]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289192 stored elements and shape (320000, 461488)>
  Coords	Values
  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171378)	0.2805816206356074
  (0, 271016)	0.45356623916588285
  (0, 279082)	0.17825180109103442
  (0, 388348)	0.2198507607206174
  (0, 398906)	0.34910438732642673
  (0, 409143)	0.3143047059807971
  (0, 420984)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217562)	0.40288153995289894
  (1, 256777)	0.28751585696559306
  (1, 348135)	0.4739279595416274
  (1, 366203)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183312)	0.5892069252021465
  (2, 256834)	0.2564939661498776
  :	:
  (319994, 443794)	0.2782185641032538


## Training the Machine learning

Logistic Regression

In [30]:
model=LogisticRegression(max_iter=1000)

In [31]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Accuracy score

In [35]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [36]:
print("Accuracy score: ",training_data_accuracy)

Accuracy score:  0.79871953125


In [38]:
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [39]:
print("Accuracy score: ",test_data_accuracy)

Accuracy score:  0.77668125


#### SAVING THE TRAINED MODEL

In [40]:
import pickle

In [43]:
filename=r"C:\Users\gacha\Downloads\trained_model.sav"
pickle.dump(model,open(filename,"wb"))

Using that saved model

In [44]:
loaded_model=pickle.load(open(filename,"rb"))

In [48]:
X_new=X_test[200]
print(Y_test[200])
predict=model.predict(X_new)
print(predict)
# print(Y_test[0])

if predict[0] == 0:
  print("Negative Tweet")
else:
  print("Positive Tweet")

1
[1]
Positive Tweet


In [50]:
X_new=X_test[3]
print(Y_test[3])
predict=model.predict(X_new)
print(predict)
# print(Y_test[0])

if predict[0] == 0:
  print("Negative Tweet")
else:
  print("Positive Tweet")

0
[0]
Negative Tweet
